# Setup

To run the example, a file named `secret.py` must be created in the notebooks
folder with the following content:

```python
# Configuration settings

# SLIPO workbench installation
BASE_URL = 'https://app.dev.slipo.eu'

# SLIPO API key
API_KEY = ''
```

The `API_KEY` value must be set to a valid SLIPO Application Key. The file must be imported before creating a new context:

```python
from secret import BASE_URL, API_KEY
```

In [ ]:
# Create new context
from slipoframes.context import SlipoContext

from secret import BASE_URL, API_KEY

ctx = SlipoContext(
    base_url = BASE_URL,
    api_key = API_KEY,
    requires_ssl = False
)

# Transform operation

Next we are going to:

* Upload the file `DKV_Fuel_Berlin.csv` from the local folder `datasets` to the remote folder `notebooks/datasets`. The remote folder will be created automatically if not already exists. The option `overwrite` is also set to `True` to overwrite any existing files.
* Upload the contents of folder `config` from the local file system to the remote folder `notebooks/config`.
* Execute a transform operation to convert the `CSV` data to `N-Triples`.
* Check the status of the operation

Examples of mapping and classification files can be found in folders [test/conf](https://github.com/SLIPO-EU/TripleGeo/tree/master/test/conf) and [test/classification](https://github.com/SLIPO-EU/TripleGeo/tree/master/test/classification) of the TripleGeo project.

In [ ]:
# Upload file DKV_Berlin.csv
ctx.file_upload('./datasets/DKV_Fuel_Berlin.csv', 'notebooks/datasets/DKV_Fuel_Berlin.csv', overwrite=True)

In [ ]:
# Upload all files in the config folder
ctx.file_upload('./config', 'notebooks/config', overwrite=True)

In [ ]:
# Browse remote user file system
df_files = ctx.file_browse(sort_col='size', format_size=True, sort_asc=False)

df_files[df_files['path'].str.startswith("notebooks")]

In [ ]:
# Convert DKV_Fuel_Berlin.csv file to N-Triples
transform = ctx.transform_csv(
    'notebooks/datasets/DKV_Fuel_Berlin.csv',
    attrCategory='all_tags',
    attrKey='ID',
    attrName='name',
    attrX='lon',
    attrY='lat',
    delimiter=';',
    featureSource='DKV',
    quote='',
    mappingSpec='notebooks/config/DKV_Fuel_Berlin_slipo_mappings.yml',
    classificationSpec='notebooks/config/DKV_POI_sample_classification.csv'
)

In [ ]:
# Check process status
transform = ctx.process_status(transform)

In [ ]:
# View all steps of the process. For a simple API operation only a single step exists
transform.steps()

In [ ]:
# View files generated by the process
transform.files()

In [ ]:
# Get default output file
transform.output()

In [ ]:
# Download output file. Use process id, version and file id from previous steps
ctx.process_file_download(transform.output(), target = './output/DKV_Fuel_Berlin.nt', overwrite=True)

# Data Processing

Next we are installing a library for processing RDF
data and execute a simple query. Currently `gastrodon` does not support Python 3.7

In [ ]:
%%capture
%pip install gastrodon

In [ ]:
# Create graph
from rdflib import Graph,URIRef
from gastrodon import LocalEndpoint,one,QName

g = Graph()
g.parse('./output/DKV_Fuel_Berlin.nt', format="nt")

In [ ]:
# Get graph size
len(g)

In [ ]:
# Create an endpoint and execute a simple query
e = LocalEndpoint(g)

properties = e.select("""
   SELECT ?p (COUNT(*) AS ?cnt) {
      ?s ?p ?o .
   } GROUP BY ?p ORDER BY DESC(?cnt)
""")

properties